In [5]:
import requests #імпортуємо бібліотеку, яка дозволяє нам робити запити та працювати із АРІ
from pprint import pprint as pp # pprint - бібліотека, яка дозволяє зручно переглядати ієрархічні дані
import sqlite3 # бібліотека для створення та роботи із sqlite базами даних
import pandas as pd #бібліотека для роботи із масивами та таблицями
from tqdm import tqdm # бібліотека для створення прогресбарів
from datetime import date #бібліотека для роботи із часом
# import seaborn as sns #бібліотека для візуалізації даних
import os

from pymongo import MongoClient
from IPython.display import clear_output 


### Куди копати

* Аналіз допорогових закупівель
* Активний аукціон система пропонує постачальників, система пінгує
* Ковідні закупівлі - окрема тема

## Download data from API

Depricated: Use scrapy

In [3]:
import datetime


datetime.datetime.fromisoformat('2021-10-01') - datetime.timedelta(days=30)

datetime.datetime(2021, 9, 1, 0, 0)

In [37]:
from urllib.parse import urlparse, parse_qsl, urljoin
import datetime

tz = datetime.timezone(datetime.timedelta(seconds=10800))
start_date = datetime.datetime(2021,1,1,tzinfo=tz)
end_date = datetime.datetime(2021,10,1,tzinfo=tz)

api_path = "https://public.api.openprocurement.org/api/2.5/"

next_page_url = urljoin(api_path,f"tenders?offset={start_date.isoformat()}")
offset_date = start_date

tender_ids = []

while offset_date<end_date:

    clear_output()
    print(f'Date {offset_date} tender ids {len(tender_ids) }')

    page = requests.get(next_page_url).json()
    
    # new next page
    next_page_url = page["next_page"]["uri"]    

    tender_ids += [tender["id"] for tender in page["data"]]

    qs = dict(
        parse_qsl(
            urlparse(next_page_url).query
            )
        )

    # to compare with end date
    offset_date = datetime.datetime.fromisoformat(qs['offset'])

# unique
tender_ids = list(set(tender_ids))

Date 2021-03-17 13:53:34.857358+02:00 tender ids 1002100


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
with open('./scraper/tender_ids.txt', 'w') as f:
    f.writelines(s + '\n' for s in tender_ids)

In [163]:
with open('./scraper/tender_ids.txt', 'r') as f:
    tender_ids = [line.strip() for line in f.readlines()]

len(tender_ids)

476400

In [19]:
client = MongoClient()
db = client['prozorro']
tenders_collection = db['tenders']

# USE SPIDER - work faster and has outothrottle mechanism
def get_tender(tender_id):
    response = requests.get(
        urljoin(api_path+'tenders/', tender_id)
    ).json()

    tender = response["data"]

    del tender['id']
    
    update_info = tenders_collection.update({'_id':tender_id}, tender, upsert=True)
    return update_info

# slow
for tender_id in tqdm(tender_ids):
    
    get_tender(tender_id)


In [172]:
import concurrent.futures

# faster

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     for tender_id, prime in zip(tender_ids, executor.map(get_tender, tender_ids)):
#         clear_output()
#         print('Tender %s is acknowledged: %s' % (tender_id, prime))

## Get data from local DB

In [6]:
client = MongoClient()
db = client['prozorro']
tenders_collection = db['tenders']

In [7]:
# https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/#return-specific-fields-in-embedded-documents

def read_mongo(db, collection, query={}, fields=None, id_to_index=False):
    """ Read from Mongo and Store into DataFrame """

    # Make a query to the specific DB and Collection
    
    # filter fields
    _fields = None
    if fields is not None:
        _fields = fields if isinstance(fields, dict) else {field:1 for field in fields}

    cursor = db[collection].find(query, _fields)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if id_to_index:
        df.set_index('_id')

    return df

# You can put filter here or later in pandas dataset
# equvalent to
# SELECT fields
#   FROM collection
#   WHERE query
df = read_mongo(db, collection='tenders', query={'status':'complete'}, fields=['status', 'items'])


In [17]:
df.columns

Index(['_id', 'status', 'items'], dtype='object')

In [21]:
items = []

for row in df[:100].itertuples():
    for itm in row.items:
        # itm._id
        items.append(itm)

In [22]:
pd.DataFrame(items)

,id,description,quantity,classification,unit,deliveryDate,deliveryAddress,additionalClassifications,deliveryLocation
0,9a63616c7ce84a26a8743d974eee647e,Електрична енергія з оплатою розподілу через п...,194000.0,"{'description': 'Електрична енергія', 'scheme'...","{'name': 'кВт⋅год', 'code': 'KWH'}","{'startDate': '2021-01-01T00:00:00+02:00', 'en...","{'streetAddress': 'Курчатова 11', 'locality': ...",NaN,NaN
1,e9309354b5994fca8962019ad36c0856,Електрична енергія з оплатою розподілу через п...,435000.0,"{'description': 'Електрична енергія', 'scheme'...","{'name': 'кВт⋅год', 'code': 'KWH'}","{'startDate': '2021-01-01T00:00:00+02:00', 'en...","{'streetAddress': 'Геофізиків, 10', 'locality'...",NaN,NaN
2,131afb8d63b4bc56c9adcbb1872b2249,бюлетні,326.0,"{'description': 'Бланки', 'scheme': 'ДК021', '...","{'name': 'шт', 'code': 'H87', 'value': {'amoun...","{'startDate': '2020-12-29T10:01:46+02:00', 'en...","{'streetAddress': 'вул. Завгороднього, 9', 'lo...",NaN,NaN
3,5d9d6faedfed69264968b4ac1676bbba,протоколи,20.0,"{'description': 'Бланки', 'scheme': 'ДК021', '...","{'name': 'шт', 'code': 'H87', 'value': {'amoun...","{'startDate': '2020-12-29T10:01:46+02:00', 'en...","{'streetAddress': 'вул. Завгороднього, 9', 'lo...",NaN,NaN
4,af6856d348cf4ca694924fbbf8b74edb,"Експертиза робочого проекту ""Капітальний ремон...",1.0,"{'description': 'Експертні послуги', 'scheme':...","{'name': 'послуга', 'code': 'E48'}",{'endDate': '2020-12-31T00:00:00+02:00'},"{'streetAddress': '1 Травня ,7а', 'locality': ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
200,cf2f0af32aa640548259cb252b515351,продукти різні ...,30.0,{'description': 'Продукти харчування та сушені...,"{'name': 'кілограми', 'code': 'KGM'}","{'startDate': '2020-12-17T00:00:00+02:00', 'en...","{'streetAddress': 'ІФранка 42', 'locality': 'Ч...",NaN,NaN
201,5c47d6052eaa4117ab3df3067c2e6ed4,Геодезична зйомка земельної ділянки по вул. Ва...,1.0,"{'description': 'Архітектурні, інженерні та ге...","{'name': 'послуга', 'code': 'E48'}",{'endDate': '2021-12-31T00:00:00+02:00'},"{'streetAddress': 'просп. Московський, 96-А', ...",NaN,NaN
202,b5058d77061db798e0cdf4a0b107c853,вапно 4 кг,20.0,"{'description': 'Вапняк, гіпс і крейда', 'sche...","{'name': 'шт', 'code': 'H87'}",{'endDate': '2020-12-31T18:01:02+02:00'},"{'streetAddress': 'вул. Винниченка, 4', 'local...",NaN,NaN
203,be4dc09e865444f1a39674ef0769d29c,запасні частини ...,9.0,{'description': 'Запасні частини до вантажних ...,"{'name': 'штуки', 'code': 'H87'}","{'startDate': '2020-12-10T00:00:00+02:00', 'en...","{'streetAddress': 'ІФранка 42', 'locality': 'Ч...",NaN,NaN


**tender Schema**

[here](https://gist.github.com/imaginal/bf19e659b34b6acd7ee002f6b88dbf81)

interesting example

https://public.api.openprocurement.org/api/2.5/tenders/98d54c150056474fbedb5420f8aad0d9